In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
!pip install sweetviz
import sweetviz as sv

Mounted at /content/drive
     |████████████████████████████████| 15.1MB 10.5MB/s 
     |████████████████████████████████| 81kB 5.5MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

In [4]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

In [5]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [6]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data


In [10]:
cols = list(train_qui2.iloc[:,1:-2].columns)

In [27]:
for col in cols:
  qui_train = list(train_qui2[col].value_counts().index)
  qui_test = list(test_qui2[col].value_counts().index)
  print(col, '에서 불일치')
  n = 0
  for item in qui_test:    
    if item not in qui_train:
      n +=1
  print('{}개'.format(n))

quality_1 에서 불일치
11개
quality_2 에서 불일치
286개
quality_5 에서 불일치
1860개
quality_6 에서 불일치
42개
quality_7 에서 불일치
447개
quality_8 에서 불일치
10개
quality_9 에서 불일치
225개
quality_10 에서 불일치
1690개
quality_11 에서 불일치
3개


In [24]:
###### 그냥 쓰자니 불일치가 너무 많아서... 전처럼 카테고리화 해서 확률로 전환한다.

train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

In [25]:
train_qui3

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
1,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
2,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
3,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00
828620,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00
828621,24997,0.0,0.0,2.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00
828622,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00


In [ ]:
#################################################################
#                                                                                                                                                   #
#           일단 오류를 보고한 소비자의 퀼리티 데이터와 오류를 보고하지 않은 소비자의 퀼리티 데이터 비교          #
#                                                                                                                                                   #
#################################################################

# 우선 카테고리화 하지 않은 상태로 비교해본다... 그리고 카테고리화 해서 비교해본다.

train_qui2

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time
0,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
1,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
2,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
3,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2020-11-29 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,2020-11-24 03:30:00
828620,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,2020-11-24 03:30:00
828621,24997,0.0,0.0,3.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,2020-11-24 03:30:00
828622,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,2020-11-24 03:30:00


In [ ]:
prob_list = list(set(train_prob.user_id))

In [ ]:
train_qui2['problems'] = train_qui2.user_id.isin(prob_list)
train_qui2_tr = train_qui2[train_qui2['problems']].dropna()
train_qui2_fls = train_qui2[train_qui2['problems'] != True].dropna()

In [ ]:
## T 검정 진행
from scipy.stats import ttest_ind
from scipy.stats import chisquare
from scipy import stats

In [ ]:
# T 검정을 하기 위해서 두 표본이 등분산인지 확인할 필요가 있음
# 따라서 등분산 검정을 진행
cols = list(train_qui2_tr.iloc[:,1:-2].columns)

r_list = []

for col in cols:
  result = stats.levene(train_qui2_tr[col], train_qui2_fls[col])
  r_list.append(list(result))
#print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['등분산성'] = df['P_value'].apply(lambda x: '불만족' if x < 0.05 else '만족')
df

,통계값,P_value,등분산성
quality_1,1026.039420,5.488152e-225,불만족
quality_2,5.045943,2.468394e-02,불만족
quality_5,210.015689,1.382207e-47,불만족
quality_6,27.125934,1.906712e-07,불만족
quality_7,51.825960,6.069832e-13,불만족
quality_8,0.244978,6.206340e-01,만족
quality_9,23.230138,1.437538e-06,불만족
quality_10,569.899036,6.548420e-126,불만족
quality_11,3085.176295,0.000000e+00,불만족
quality_12,28.096555,1.154420e-07,불만족


In [ ]:
# quality_8만 등분산성을 만족하고 나머지는 불만족한다..
# ttest_ind에 equal_var=False 을 넣어서 등분산 불만족을 표시해주어야 함.

r_list = []

for col in cols:
  if col == 'quality_8':
    result = ttest_ind(train_qui2_tr[col], train_qui2_fls[col])
    r_list.append(list(result))
  else:
    result = ttest_ind(train_qui2_tr[col], train_qui2_fls[col], equal_var=False)
    r_list.append(list(result))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['둘이_다른가'] = df['P_value'].apply(lambda x: '다르다' if x < 0.05 else '같다')
df

,통계값,P_value,둘이_다른가
quality_1,-31.549742,2.660366e-218,다르다
quality_2,-2.377491,1.743125e-02,다르다
quality_5,14.463984,2.075074e-47,다르다
quality_6,-6.544800,5.961198e-11,다르다
quality_7,-7.218764,5.250907e-13,다르다
quality_8,0.494952,6.206340e-01,같다
quality_9,-4.936972,7.937170e-07,다르다
quality_10,23.762820,9.036843e-125,다르다
quality_11,-55.539807,0.000000e+00,다르다
quality_12,-5.273121,1.341602e-07,다르다


In [ ]:
train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

train_qui3['problems'] = train_qui3.user_id.isin(prob_list)
train_qui3_tr = train_qui3[train_qui3['problems']]
train_qui3_fls = train_qui3[train_qui3['problems'] != True]

# T 검정을 하기 위해서 두 표본이 등분산인지 확인할 필요가 있음
# 따라서 등분산 검정을 진행
cols = list(train_qui3.iloc[:,1:-2].columns)

r_list = []

for col in cols:
  result = stats.levene(train_qui3_tr[col].dropna(), train_qui3_fls[col].dropna())
  r_list.append(list(result))
#print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['등분산성'] = df['P_value'].apply(lambda x: '불만족' if x < 0.05 else '만족')
df

,통계값,P_value,등분산성
quality_1,2717.945783,0.000000e+00,불만족
quality_2,3123.124718,0.000000e+00,불만족
quality_5,31347.129682,0.000000e+00,불만족
quality_6,2606.656698,0.000000e+00,불만족
quality_7,220.687432,6.499446e-50,불만족
quality_8,819.935395,3.062387e-180,불만족
quality_9,501.722687,4.327639e-111,불만족
quality_10,17489.210820,0.000000e+00,불만족
quality_11,2452.525667,0.000000e+00,불만족
quality_12,81.884550,1.445719e-19,불만족


In [ ]:
# 카테고리화 한 경우는 전부 불만족이다.
# ttest_ind에 equal_var=False 을 넣어서 등분산 불만족을 표시해주어야 함.

r_list = []

for col in cols:
  result = ttest_ind(train_qui3_tr[col].dropna(), train_qui3_fls[col].dropna(), equal_var=False)
  r_list.append(list(result))

df = pd.DataFrame(r_list, index = cols, columns = ['통계값', 'P_value'])
df['둘이_다른가'] = df['P_value'].apply(lambda x: '다르다' if x < 0.05 else '같다')
df

,통계값,P_value,둘이_다른가
quality_1,-46.732296,0.000000e+00,다르다
quality_2,-52.324643,0.000000e+00,다르다
quality_5,150.575219,0.000000e+00,다르다
quality_6,-45.514973,0.000000e+00,다르다
quality_7,14.749354,3.153937e-49,다르다
quality_8,28.379437,4.475413e-177,다르다
quality_9,-22.539983,1.820894e-112,다르다
quality_10,201.287485,0.000000e+00,다르다
quality_11,-49.995806,0.000000e+00,다르다
quality_12,-9.059933,1.308068e-19,다르다


In [ ]:
train_qui3_tr

,user_id,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time,problems
120,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
121,10004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
122,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
123,10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
124,10004,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2020-11-02 23:20:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828620,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828621,24997,0.0,0.0,2.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
828622,24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,2020-11-24 03:30:00,True
